In [12]:
import os
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('default')
import pandas as pd
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower
plt.tight_layout()
import matplotlib 
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 20}
matplotlib.rc('font', **font)

<Figure size 1200x800 with 0 Axes>

In [13]:
# Read the origin file
original_df = pd.read_csv("../data/penguins_lter.csv")
original_df

,studyName,Sample Number,Species,Region,Island,Stage,Individual ID,Clutch Completion,Date Egg,Culmen Length (mm),Culmen Depth (mm),Flipper Length (mm),Body Mass (g),Sex,Delta 15 N (o/oo),Delta 13 C (o/oo),Comments
0,PAL0708,1,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N1A1,Yes,11-11-07,39.1,18.7,181.0,3750.0,MALE,NaN,NaN,Not enough blood for isotopes.
1,PAL0708,2,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N1A2,Yes,11-11-07,39.5,17.4,186.0,3800.0,FEMALE,8.94956,-24.69454,NaN
2,PAL0708,3,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N2A1,Yes,11/16/07,40.3,18.0,195.0,3250.0,FEMALE,8.36821,-25.33302,NaN
3,PAL0708,4,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N2A2,Yes,11/16/07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Adult not sampled.
4,PAL0708,5,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N3A1,Yes,11/16/07,36.7,19.3,193.0,3450.0,FEMALE,8.76651,-25.32426,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339,PAL0910,120,Gentoo penguin (Pygoscelis papua),Anvers,Biscoe,"Adult, 1 Egg Stage",N38A2,No,12-01-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
340,PAL0910,121,Gentoo penguin (Pygoscelis papua),Anvers,Biscoe,"Adult, 1 Egg Stage",N39A1,Yes,11/22/09,46.8,14.3,215.0,4850.0,FEMALE,8.41151,-26.13832,NaN
341,PAL0910,122,Gentoo penguin (Pygoscelis papua),Anvers,Biscoe,"Adult, 1 Egg Stage",N39A2,Yes,11/22/09,50.4,15.7,222.0,5750.0,MALE,8.30166,-26.04117,NaN
342,PAL0910,123,Gentoo penguin (Pygoscelis papua),Anvers,Biscoe,"Adult, 1 Egg Stage",N43A1,Yes,11/22/09,45.2,14.8,212.0,5200.0,FEMALE,8.24246,-26.11969,NaN


In [14]:
# Number of unique values in each columns
print(original_df.nunique())

studyName                3
Sample Number          152
Species                  3
Region                   1
Island                   3
Stage                    1
Individual ID          190
Clutch Completion        2
Date Egg                50
Culmen Length (mm)     164
Culmen Depth (mm)       80
Flipper Length (mm)     55
Body Mass (g)           94
Sex                      3
Delta 15 N (o/oo)      330
Delta 13 C (o/oo)      331
Comments                 7
dtype: int64


In [15]:
# Each study researches only one kind of penguin. We want to predict types of penguins based on natural characteristics
df_remove_spare_columns = original_df.drop('studyName', inplace=False, axis=1)
# studyName is removed, its sample order should be too
df_remove_spare_columns = df_remove_spare_columns.drop('Sample Number', inplace=False, axis=1)
# all stages are the same
df_remove_spare_columns = df_remove_spare_columns.drop('Stage', inplace=False, axis=1)
# all regions are the same
df_remove_spare_columns = df_remove_spare_columns.drop('Region', inplace=False, axis=1)
# Date Egg may reveals the species of the corresponding research
df_remove_spare_columns = df_remove_spare_columns.drop('Date Egg', inplace=False, axis=1)
# Individual ID is subjective and distinct with each species
df_remove_spare_columns = df_remove_spare_columns.drop('Individual ID', inplace=False, axis=1)
# Comments are personal and has too many NaN values
df_remove_spare_columns = df_remove_spare_columns.drop('Comments', inplace=False, axis=1)

In [16]:
df_remove_spare_columns

,Species,Island,Clutch Completion,Culmen Length (mm),Culmen Depth (mm),Flipper Length (mm),Body Mass (g),Sex,Delta 15 N (o/oo),Delta 13 C (o/oo)
0,Adelie Penguin (Pygoscelis adeliae),Torgersen,Yes,39.1,18.7,181.0,3750.0,MALE,NaN,NaN
1,Adelie Penguin (Pygoscelis adeliae),Torgersen,Yes,39.5,17.4,186.0,3800.0,FEMALE,8.94956,-24.69454
2,Adelie Penguin (Pygoscelis adeliae),Torgersen,Yes,40.3,18.0,195.0,3250.0,FEMALE,8.36821,-25.33302
3,Adelie Penguin (Pygoscelis adeliae),Torgersen,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Adelie Penguin (Pygoscelis adeliae),Torgersen,Yes,36.7,19.3,193.0,3450.0,FEMALE,8.76651,-25.32426
...,...,...,...,...,...,...,...,...,...,...
339,Gentoo penguin (Pygoscelis papua),Biscoe,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN
340,Gentoo penguin (Pygoscelis papua),Biscoe,Yes,46.8,14.3,215.0,4850.0,FEMALE,8.41151,-26.13832
341,Gentoo penguin (Pygoscelis papua),Biscoe,Yes,50.4,15.7,222.0,5750.0,MALE,8.30166,-26.04117
342,Gentoo penguin (Pygoscelis papua),Biscoe,Yes,45.2,14.8,212.0,5200.0,FEMALE,8.24246,-26.11969


## Warning: The code below can change outside files

In [17]:
# Only use below code when needing to save the data
df_remove_spare_columns.to_csv('../data/data_remove_spare_columns.csv', index = False)
df_remove_spare_columns

,Species,Island,Clutch Completion,Culmen Length (mm),Culmen Depth (mm),Flipper Length (mm),Body Mass (g),Sex,Delta 15 N (o/oo),Delta 13 C (o/oo)
0,Adelie Penguin (Pygoscelis adeliae),Torgersen,Yes,39.1,18.7,181.0,3750.0,MALE,NaN,NaN
1,Adelie Penguin (Pygoscelis adeliae),Torgersen,Yes,39.5,17.4,186.0,3800.0,FEMALE,8.94956,-24.69454
2,Adelie Penguin (Pygoscelis adeliae),Torgersen,Yes,40.3,18.0,195.0,3250.0,FEMALE,8.36821,-25.33302
3,Adelie Penguin (Pygoscelis adeliae),Torgersen,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Adelie Penguin (Pygoscelis adeliae),Torgersen,Yes,36.7,19.3,193.0,3450.0,FEMALE,8.76651,-25.32426
...,...,...,...,...,...,...,...,...,...,...
339,Gentoo penguin (Pygoscelis papua),Biscoe,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN
340,Gentoo penguin (Pygoscelis papua),Biscoe,Yes,46.8,14.3,215.0,4850.0,FEMALE,8.41151,-26.13832
341,Gentoo penguin (Pygoscelis papua),Biscoe,Yes,50.4,15.7,222.0,5750.0,MALE,8.30166,-26.04117
342,Gentoo penguin (Pygoscelis papua),Biscoe,Yes,45.2,14.8,212.0,5200.0,FEMALE,8.24246,-26.11969


In [18]:
df_remove_spare_columns = pd.read_csv("../data/data_remove_spare_columns.csv")
df_remove_spare_columns

,Species,Island,Clutch Completion,Culmen Length (mm),Culmen Depth (mm),Flipper Length (mm),Body Mass (g),Sex,Delta 15 N (o/oo),Delta 13 C (o/oo)
0,Adelie Penguin (Pygoscelis adeliae),Torgersen,Yes,39.1,18.7,181.0,3750.0,MALE,NaN,NaN
1,Adelie Penguin (Pygoscelis adeliae),Torgersen,Yes,39.5,17.4,186.0,3800.0,FEMALE,8.94956,-24.69454
2,Adelie Penguin (Pygoscelis adeliae),Torgersen,Yes,40.3,18.0,195.0,3250.0,FEMALE,8.36821,-25.33302
3,Adelie Penguin (Pygoscelis adeliae),Torgersen,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Adelie Penguin (Pygoscelis adeliae),Torgersen,Yes,36.7,19.3,193.0,3450.0,FEMALE,8.76651,-25.32426
...,...,...,...,...,...,...,...,...,...,...
339,Gentoo penguin (Pygoscelis papua),Biscoe,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN
340,Gentoo penguin (Pygoscelis papua),Biscoe,Yes,46.8,14.3,215.0,4850.0,FEMALE,8.41151,-26.13832
341,Gentoo penguin (Pygoscelis papua),Biscoe,Yes,50.4,15.7,222.0,5750.0,MALE,8.30166,-26.04117
342,Gentoo penguin (Pygoscelis papua),Biscoe,Yes,45.2,14.8,212.0,5200.0,FEMALE,8.24246,-26.11969


In [19]:
# Outlier is defined as a sample with no valid important values
df_remove_outliers = df_remove_spare_columns[df_remove_spare_columns['Culmen Length (mm)'].notna() |
                                            df_remove_spare_columns['Culmen Depth (mm)'].notna() |
                                            df_remove_spare_columns['Flipper Length (mm)'].notna() |
                                            df_remove_spare_columns['Body Mass (g)'].notna() |
                                            df_remove_spare_columns['Delta 15 N (o/oo)'].notna() |
                                            df_remove_spare_columns['Delta 13 C (o/oo)'].notna()]
df_remove_outliers.to_csv('../data/data_remove_outliers.csv', index = False)

,Species,Island,Clutch Completion,Culmen Length (mm),Culmen Depth (mm),Flipper Length (mm),Body Mass (g),Sex,Delta 15 N (o/oo),Delta 13 C (o/oo)
0,Adelie Penguin (Pygoscelis adeliae),Torgersen,Yes,39.1,18.7,181.0,3750.0,MALE,NaN,NaN
1,Adelie Penguin (Pygoscelis adeliae),Torgersen,Yes,39.5,17.4,186.0,3800.0,FEMALE,8.94956,-24.69454
2,Adelie Penguin (Pygoscelis adeliae),Torgersen,Yes,40.3,18.0,195.0,3250.0,FEMALE,8.36821,-25.33302
4,Adelie Penguin (Pygoscelis adeliae),Torgersen,Yes,36.7,19.3,193.0,3450.0,FEMALE,8.76651,-25.32426
5,Adelie Penguin (Pygoscelis adeliae),Torgersen,Yes,39.3,20.6,190.0,3650.0,MALE,8.66496,-25.29805
...,...,...,...,...,...,...,...,...,...,...
338,Gentoo penguin (Pygoscelis papua),Biscoe,No,47.2,13.7,214.0,4925.0,FEMALE,7.99184,-26.20538
340,Gentoo penguin (Pygoscelis papua),Biscoe,Yes,46.8,14.3,215.0,4850.0,FEMALE,8.41151,-26.13832
341,Gentoo penguin (Pygoscelis papua),Biscoe,Yes,50.4,15.7,222.0,5750.0,MALE,8.30166,-26.04117
342,Gentoo penguin (Pygoscelis papua),Biscoe,Yes,45.2,14.8,212.0,5200.0,FEMALE,8.24246,-26.11969


In [85]:
df_remove_outliers = pd.read_csv("../data/data_remove_outliers.csv")
df_remove_outliers

,Species,Island,Clutch Completion,Culmen Length (mm),Culmen Depth (mm),Flipper Length (mm),Body Mass (g),Sex,Delta 15 N (o/oo),Delta 13 C (o/oo)
0,Adelie Penguin (Pygoscelis adeliae),Torgersen,Yes,39.1,18.7,181.0,3750.0,MALE,NaN,NaN
1,Adelie Penguin (Pygoscelis adeliae),Torgersen,Yes,39.5,17.4,186.0,3800.0,FEMALE,8.94956,-24.69454
2,Adelie Penguin (Pygoscelis adeliae),Torgersen,Yes,40.3,18.0,195.0,3250.0,FEMALE,8.36821,-25.33302
3,Adelie Penguin (Pygoscelis adeliae),Torgersen,Yes,36.7,19.3,193.0,3450.0,FEMALE,8.76651,-25.32426
4,Adelie Penguin (Pygoscelis adeliae),Torgersen,Yes,39.3,20.6,190.0,3650.0,MALE,8.66496,-25.29805
...,...,...,...,...,...,...,...,...,...,...
337,Gentoo penguin (Pygoscelis papua),Biscoe,No,47.2,13.7,214.0,4925.0,FEMALE,7.99184,-26.20538
338,Gentoo penguin (Pygoscelis papua),Biscoe,Yes,46.8,14.3,215.0,4850.0,FEMALE,8.41151,-26.13832
339,Gentoo penguin (Pygoscelis papua),Biscoe,Yes,50.4,15.7,222.0,5750.0,MALE,8.30166,-26.04117
340,Gentoo penguin (Pygoscelis papua),Biscoe,Yes,45.2,14.8,212.0,5200.0,FEMALE,8.24246,-26.11969


In [38]:
print(df_remove_spare_columns.nunique())

Species                  3
Island                   3
Clutch Completion        2
Culmen Length (mm)     164
Culmen Depth (mm)       80
Flipper Length (mm)     55
Body Mass (g)           94
Sex                      3
Delta 15 N (o/oo)      330
Delta 13 C (o/oo)      331
dtype: int64


In [68]:
df_Adelie = df_remove_spare_columns[df_remove_spare_columns['Species'] == 'Adelie Penguin (Pygoscelis adeliae)']
df_Chinstrap = df_remove_spare_columns[df_remove_spare_columns['Species'] == 'Chinstrap penguin (Pygoscelis antarctica)']
df_Gentoo = df_remove_spare_columns[df_remove_spare_columns['Species'] == 'Gentoo penguin (Pygoscelis papua)']

# Suffle
df_Adelie = df_Adelie.sample(frac=1)
df_Chinstrap = df_Chinstrap.sample(frac=1)
df_Gentoo = df_Gentoo.sample(frac=1)

In [69]:
def dfGetTrainData(df, percentage):
    df_len = len(df)
    return df.head(int(df_len * percentage))
    
def dfGetTestData(df, percentage):
    df_len = len(df)    
    return df.tail(df_len - int(df_len * percentage))

In [70]:
percentage = 0.8

df_Adelie_train = dfGetTrainData(df_Adelie, percentage)
df_Chinstrap_train = dfGetTrainData(df_Chinstrap, percentage)
df_Gentoo_train = dfGetTrainData(df_Gentoo, percentage)

df_Adelie_test = dfGetTestData(df_Adelie, percentage)
df_Chinstrap_test = dfGetTestData(df_Chinstrap, percentage)
df_Gentoo_test = dfGetTestData(df_Gentoo, percentage)

In [71]:
frames = [df_Adelie_train, df_Chinstrap_train, df_Gentoo_train]
df_train = pd.concat(frames).sample(frac=1)
frames = [df_Adelie_test, df_Chinstrap_test, df_Gentoo_test]
df_test = pd.concat(frames).sample(frac=1)

In [72]:
try:
    os.makedirs('../data/data1')
except OSError as e:
    pass
df_train.to_csv('../data/data1/train_data.csv', index = False)

## Warning: The function below can change outside files

In [62]:
def splitTrainTest(df, percentage, data_name):
    df_Adelie = df_remove_spare_columns[df_remove_spare_columns['Species'] == 'Adelie Penguin (Pygoscelis adeliae)']
    df_Chinstrap = df_remove_spare_columns[df_remove_spare_columns['Species'] == 'Chinstrap penguin (Pygoscelis antarctica)']
    df_Gentoo = df_remove_spare_columns[df_remove_spare_columns['Species'] == 'Gentoo penguin (Pygoscelis papua)']

    # Suffle
    df_Adelie = df_Adelie.sample(frac=1)
    df_Chinstrap = df_Chinstrap.sample(frac=1)
    df_Gentoo = df_Gentoo.sample(frac=1)
    
    def dfGetTrainData(df, percentage):
        df_len = len(df)
        return df.head(int(df_len * percentage))

    def dfGetTestData(df, percentage):
        df_len = len(df)    
        return df.tail(df_len - int(df_len * percentage))
    
    df_Adelie_train = dfGetTrainData(df_Adelie, percentage)
    df_Chinstrap_train = dfGetTrainData(df_Chinstrap, percentage)
    df_Gentoo_train = dfGetTrainData(df_Gentoo, percentage)

    df_Adelie_test = dfGetTestData(df_Adelie, percentage)
    df_Chinstrap_test = dfGetTestData(df_Chinstrap, percentage)
    df_Gentoo_test = dfGetTestData(df_Gentoo, percentage)
    
    frames = [df_Adelie_train, df_Chinstrap_train, df_Gentoo_train]
    df_train = pd.concat(frames).sample(frac=1)
    frames = [df_Adelie_test, df_Chinstrap_test, df_Gentoo_test]
    df_test = pd.concat(frames).sample(frac=1)
    
    try:
        os.makedirs('../data/' + data_name)
    except OSError as e:
        pass
    
    df_train.to_csv('../data/' + data_name + '/train_data.csv', index = False)
    df_test.to_csv('../data/' + data_name + '/test_data.csv', index = False)

In [63]:
splitTrainTest(df_remove_outliers, percentage = 0.8, data_name = 'data1')

In [64]:
train_data = pd.read_csv("../data/data1/train_data.csv")
test_data = pd.read_csv("../data/data1/test_data.csv")

In [65]:
df_remove_outliers.nunique()

Species                  3
Island                   3
Clutch Completion        2
Culmen Length (mm)     164
Culmen Depth (mm)       80
Flipper Length (mm)     55
Body Mass (g)           94
Sex                      3
Delta 15 N (o/oo)      330
Delta 13 C (o/oo)      331
dtype: int64

In [43]:
X_train = train_data.drop(['Species'], axis = 1)
y_train = test_data[['Species']].copy()
X_train.to_csv('../data/' + 'data1' + '/X_train.csv', index = False)
y_train.to_csv('../data/' + 'data1' + '/y_train.csv', index = False)

In [44]:
pd.read_csv("../data/data1/X_train.csv").values

array([['Biscoe', 'Yes', 46.1, ..., 'FEMALE', 7.993, -25.5139],
       ['Dream', 'No', 37.0, ..., 'FEMALE', 8.61651, -26.07021],
       ['Dream', 'Yes', 32.1, ..., 'FEMALE', 8.79787, -26.61075],
       ...,
       ['Dream', 'Yes', 46.1, ..., 'FEMALE', 8.85664, -24.55644],
       ['Torgersen', 'Yes', 42.1, ..., 'MALE', 9.05736, -25.81513],
       ['Biscoe', 'Yes', 45.2, ..., 'FEMALE', 8.24694, -26.65359]],
      dtype=object)

In [24]:
y_train

,Species,Island
0,Gentoo penguin (Pygoscelis papua),Biscoe
1,Gentoo penguin (Pygoscelis papua),Biscoe
2,Gentoo penguin (Pygoscelis papua),Biscoe
3,Adelie Penguin (Pygoscelis adeliae),Dream
4,Chinstrap penguin (Pygoscelis antarctica),Dream
...,...,...
65,Gentoo penguin (Pygoscelis papua),Biscoe
66,Gentoo penguin (Pygoscelis papua),Biscoe
67,Adelie Penguin (Pygoscelis adeliae),Dream
68,Adelie Penguin (Pygoscelis adeliae),Dream


In [27]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
# read in the iris data
iris = load_iris()

# create X (features) and y (label)
X = iris.data
y = iris.target

print(type(X), type(y))

# use train/test split with different random_state values
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=4)

clf = GaussianNB()
clf.fit(X_train, y_train)
# GaussianNB(priors=None)

# predict our sample
y_pred = clf.predict(X_test)

# evaluation our prediction
print(metrics.accuracy_score(y_test, y_pred))
# 0.973684210526

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
0.9736842105263158


In [33]:
load_iris()

{'data': array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
  